# NLP learning for bot (conservative twitter bot)

In [1]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import pickle
import random
import seaborn as cb
import json
import numpy as np
import nltk 
from mosestokenizer import MosesDetokenizer 
from nltk import trigrams
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
import re
import matplotlib.pyplot as plt
import itertools
import collections

## Data scrapping

In [2]:
tweets_con = []

#scrape data and append tweets to list
for i, tweet in enumerate(sntwitter.TwitterSearchScraper('#conservative').get_items()): # declare a username
  if i>50000: #number of tweets you want to scrape
    break
  tweets_con.append([tweet.content]) # declare the attributes to be returned

# creating a dataframe from the list
tweets_con_df = pd.DataFrame(tweets_con, columns=['Text'])

C:\Users\djona\AppData\Local\Temp\ipykernel_9660\2216374768.py:7: FutureWarning: content is deprecated, use rawContent instead
  tweets_con.append([tweet.content]) # declare the attributes to be returned
Tweet 1575223201269006336 contains an app icon medium key '4_1594606217359462401' on app 'iphone_app'/'1142951331', but the corresponding medium is missing; dropping
Tweet 1575223201269006336 contains an app icon medium key '4_1594606217359462401' on app 'ipad_app'/'1142951331', but the corresponding medium is missing; dropping


### Creating json of tweets for preservation

In [3]:
file = open("tweet_con.json",'w')
j = json.dumps(tweets_con, default = str)
file.write(j)
file.close()

### Clean up

In [4]:
def clean_text(txt):
    cleaned = re.sub(r"http\S+","", txt)
    return cleaned

In [5]:
cleaned_text = [clean_text(elements) for elements in tweets_con_df.Text]
tt = TweetTokenizer()
normalize = [tt.tokenize(elements.lower()) for elements in cleaned_text]

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\djona\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
#stop_words = set(stopwords.words('english'))
#stop_words.add('#conservative')

# remove stop words and punctuation
#complete = [[w for w in words if not w in stop_words if w.isalnum()] for words in normalize]
complete = [[w for w in words if w.isalnum()] for words in normalize]

### Trigram splitting

In [8]:
trigram = [list(trigrams(words)) for words in complete]
trigramC = list(itertools.chain(*trigram))
tokenC = list(itertools.chain(*complete))
token_counter = collections.Counter(tokenC)
trigram_counter = collections.Counter(trigramC)
trigram_total = sum(trigram_counter.values())

In [9]:
trigram_rel_freq = {k: (v / trigram_total)+1 for (k,v) in trigram_counter.items()}
with open("models/trigram_tweet.pkl","wb") as outfile:
    pickle.dump(trigram_rel_freq, outfile)

### Generator based on model

In [10]:
with open("models/trigram_tweet.pkl", "rb") as infile:
    trigram_tweet = pickle.load(infile)

In [11]:
def trigram_gen(seed):
    trigrams = trigram_tweet
    tgms = {k: v for (k,v) in trigram_tweet.items() if k[:2] == seed}
    wds = [e[2] for e in tgms.keys()]
    if wds:
        weights = [float(e) for e in tgms.values()]
        return random.choices(population=wds,weights=weights)[0]
    else:
        w = random.choice(list(token_counter.keys()))
        return w
    


In [12]:
line = list(random.choice(list(trigram_tweet.keys())))
for i in range(30):
    w = trigram_gen((line[-2], line[-1]))
    line.append(w)
